# SQL Case Study: Country Club

### Part 1: completed in PHPMyAdmin:

#### 

/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

Answer:
SELECT name
FROM `Facilities`
WHERE membercost >0
LIMIT 0 , 30


/* Q2: How many facilities do not charge a fee to members? */

Answer:
SELECT COUNT( * )
FROM `Facilities`
WHERE membercost =0


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

Answer:
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost >0
AND membercost < ( 0.2 * monthlymaintenance )
LIMIT 0 , 30

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

Answer:
SELECT *
FROM Facilities
WHERE facid
IN ( 1, 5 )
LIMIT 0 , 30

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

Answer:
SELECT name,
CASE WHEN monthlymaintenance >100
THEN 'expensive'
ELSE 'cheap'
END AS 'monthly cost'
FROM Facilities
LIMIT 0 , 30

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

Answer:
SELECT firstname, surname
FROM Members
WHERE joindate = (
SELECT MAX( joindate )
FROM Members )

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

Answer: 

SELECT Facilities.name AS Court, CONCAT(Members.firstname, ' ', Members.surname) AS Member
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
WHERE Facilities.name IN
	(SELECT name
	FROM Facilities
	WHERE name IN ('Tennis Court 1', 'Tennis Court 2'))
AND Members.memid > 0
GROUP BY Court, Member
ORDER BY Member
;



/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

Answer: 
SELECT Facilities.name AS 'Facility', CONCAT(Members.firstname, ' ', Members.surname) AS 'Member',
CASE WHEN Bookings.memid =0
THEN Facilities.guestcost * Bookings.slots
ELSE Facilities.membercost * Bookings.slots
END AS 'Cost'
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
WHERE Bookings.starttime LIKE  '2012-09-14%'
AND (((Bookings.memid =0) AND (Facilities.guestcost * Bookings.slots >30))
OR ((Bookings.memid !=0) AND (Facilities.membercost * Bookings.slots >30)))
ORDER BY Cost DESC;


/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT * FROM (
    SELECT Facilities.name AS 'Facility', CONCAT(Members.firstname, ' ', Members.surname) AS 'Member',
	CASE WHEN Bookings.memid =0
	THEN Facilities.guestcost * Bookings.slots
	ELSE Facilities.membercost * Bookings.slots
	END AS 'Cost'
	FROM Bookings
	INNER JOIN Facilities
	ON Bookings.facid = Facilities.facid
	INNER JOIN Members
	ON Bookings.memid = Members.memid
	WHERE Bookings.starttime LIKE  '2012-09-14%') AS subquery
WHERE Cost > 30
ORDER BY Cost DESC;


### Part 2: completed in Jupyter Notebook:

In [1]:
import sqlite3
from sqlite3 import Error
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect

In [3]:
#/* Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! */

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
          SELECT * FROM
(SELECT Facilities.name AS Facility,
	SUM(CASE WHEN Bookings.memid =0
	THEN Facilities.guestcost * Bookings.slots
	ELSE Facilities.membercost * Bookings.slots
	END) AS Rev    
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
GROUP BY Facility) AS sub
WHERE sub.Rev < 1000
ORDER BY sub.Rev
;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [11]:
#/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT  (Memtable.surname || ', '|| Memtable.firstname) AS Member, (Rectable.surname || ', ' || Rectable.firstname) AS 'Recommended By'
        FROM Members AS Memtable
        LEFT JOIN Members AS Rectable ON Memtable.recommendedby = Rectable.memid
        WHERE Rectable.memid != 0
        ORDER BY Memtable.surname, Memtable.firstname
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Jack', 'Smith, Darren')
('Stibbons, Ponder', 'Tracy, Burton')
('Worthington-Smyth, Henry', 'Smith, Tracy')


In [12]:
#/* Q12: Find the facilities with their usage by member, but not guests */

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT Facilities.name AS Facility, COUNT(Bookings.bookid) AS 'Times Used by Members'
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
WHERE Members.memid >0
GROUP BY Facility
;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', 344)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Pool Table', 783)
('Snooker Table', 421)
('Squash Court', 195)
('Table Tennis', 385)
('Tennis Court 1', 308)
('Tennis Court 2', 276)


In [13]:
#/* Q13: Find the facilities usage by month, but not guests */
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT Facilities.name AS Facility, 
    COUNT(Bookings.bookid) AS 'Times Used by Members', 
    strftime('%Y-%m', Bookings.starttime) AS year_month
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
WHERE Members.memid >0
GROUP BY Facility, year_month
;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', 51, '2012-07')
('Badminton Court', 132, '2012-08')
('Badminton Court', 161, '2012-09')
('Massage Room 1', 77, '2012-07')
('Massage Room 1', 153, '2012-08')
('Massage Room 1', 191, '2012-09')
('Massage Room 2', 4, '2012-07')
('Massage Room 2', 9, '2012-08')
('Massage Room 2', 14, '2012-09')
('Pool Table', 103, '2012-07')
('Pool Table', 272, '2012-08')
('Pool Table', 408, '2012-09')
('Snooker Table', 68, '2012-07')
('Snooker Table', 154, '2012-08')
('Snooker Table', 199, '2012-09')
('Squash Court', 23, '2012-07')
('Squash Court', 85, '2012-08')
('Squash Court', 87, '2012-09')
('Table Tennis', 48, '2012-07')
('Table Tennis', 143, '2012-08')
('Table Tennis', 194, '2012-09')
('Tennis Court 1', 65, '2012-07')
('Tennis Court 1', 111, '2012-08')
('Tennis Court 1', 132, '2012-09')
('Tennis Court 2', 41, '2012-07')
('Tennis Court 2', 109, '2012-08')
('Tennis Court 2', 126, '2012-09')
